#### Start a simple Spark Session

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [2]:
df= spark.read.option("header",True).csv('/content/walmart_stock.csv')

In [3]:
df.take(1)

[Row(Date='2012-01-03', Open='59.970001', High='61.060001', Low='59.869999', Close='60.330002', Volume='12668800', Adj Close='52.619234999999996')]

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



#### What are the column names?

In [5]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### Print out the first 5 columns.

In [6]:
df.take(5)

[Row(Date='2012-01-03', Open='59.970001', High='61.060001', Low='59.869999', Close='60.330002', Volume='12668800', Adj Close='52.619234999999996'),
 Row(Date='2012-01-04', Open='60.209998999999996', High='60.349998', Low='59.470001', Close='59.709998999999996', Volume='9593300', Adj Close='52.078475'),
 Row(Date='2012-01-05', Open='59.349998', High='59.619999', Low='58.369999', Close='59.419998', Volume='12768200', Adj Close='51.825539'),
 Row(Date='2012-01-06', Open='59.419998', High='59.450001', Low='58.869999', Close='59.0', Volume='8069400', Adj Close='51.45922'),
 Row(Date='2012-01-09', Open='59.029999', High='59.549999', Low='58.919998', Close='59.18', Volume='6679300', Adj Close='51.616215000000004')]

#### Use describe() to learn about the DataFrame.

In [7]:
df_with_describe= df.describe()

In [8]:
df_with_describe=df_with_describe.drop('date')

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [9]:
df_with_describe.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [10]:
df_with_describe

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [11]:
df_with_HV_Ratio=df.withColumn("HV Ratio",df['High']/ df['Volume'])
df_with_HV_Ratio.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [12]:
df.orderBy(df['High'].desc()).select('Date').take(1)

[Row(Date='2015-01-13')]

#### What is the mean of the Close column?

In [13]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [14]:
from pyspark.sql.functions import max, min
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|    9994400|   10010500|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [15]:
df= spark.read.csv('/content/walmart_stock.csv',inferSchema=True,header=True)

In [16]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [17]:
from pyspark.sql.types import IntegerType
df = df.withColumn("Close", df["Close"].cast(IntegerType()))

In [18]:
from pyspark.sql.functions import count
df.filter(df['Close']<60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [19]:
((df.filter(df['High']>80).count()) / (df.count())) *100

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [20]:
df.corr('High','Volume')


-0.3384326061737161

#### What is the max High per year?

In [21]:
from pyspark.sql.functions import year
df= df.withColumn('Year',year(df['date']))

In [22]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: integer (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Year: integer (nullable = true)



In [23]:
df_year=df.select('Year','High').groupBy('Year').max()

In [24]:
df_year.drop('max(year)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [25]:
from pyspark.sql.functions import month
df= df.withColumn('Month',month(df['date']))

In [26]:
df_month=df.select('Month','Close').groupBy('Month').avg()

In [27]:
df_month=df_month.drop('avg(Month)')

In [28]:
df_month.orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|70.93069306930693|
|    2|70.77319587628865|
|    3| 71.2803738317757|
|    4| 72.5047619047619|
|    5|71.83962264150944|
|    6|71.97169811320755|
|    7|73.91588785046729|
|    8|             72.5|
|    9|71.65686274509804|
|   10| 71.0909090909091|
|   11|71.60396039603961|
|   12|72.33018867924528|
+-----+-----------------+

